In [20]:
# load required modules
import ee
ee.Initialize()
import pandas as pd
import numpy as np

In [21]:
# load leaflet interactive map
import geemap
Map = geemap.Map()

### MCD12Q2 v006 phenology product

In [9]:
# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)
    
# function for filtering image based on quality bits
def mask_pixelsall(bstart,bend):
    def maskPixels(image0):
        tmp = image0.select('QA_Detailed_1')
        quality = getQABits(tmp, bstart, bend, 'QA_Detailed_1')
        # Create a mask that filters out undesired areas
        mask = quality.eq(0).Or(quality.eq(1)).Or(quality.eq(2))
        return image0.updateMask(mask) 
    return(maskPixels)

# function for masking areas where there have not land use changes. This layer was created using the ESA cci map
def mask_image(image):
    image = image.updateMask(MakMarco.eq(1))
    return image

def calculate_sd(image):
    #image_sd = image.reproject(crs = 'SR-ORG:6974',scale = 463.3127165275).reduceResolution(ee.Reducer.stdDev(), False, 65536).reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1)
    image_sd = image.reduceResolution(reducer = ee.Reducer.stdDev(), bestEffort=False, maxPixels=117)
    return image_sd

In [3]:
# ---- load datasets
# load MODIS phenology product
modis_phenoprod = ee.ImageCollection('MODIS/006/MCD12Q2')
# load mask of unchanged forest pixels
MakMarco = ee.Image("users/marcogirardello/mask_unchanged_500m")
# grid for export to assets
# list of polygons
worldgrid = ee.FeatureCollection('users/marcogirardello/grid_export_phenology')

In [10]:
# polygon list for export grid
polygons = list(range(1, 42+1))

# ------- Greenup
#GUP = modis_phenoprod.select(['Greenup_1','QA_Detailed_1'])
# only take pixels above a certain threshold level
#GUP1 = GUP.map(mask_pixelsall(bstart=0,bend=1)).select('Greenup_1')
# only consider pixels where there have been no land cover changes
#GUP2 = GUP1.map(mask_image)
# calculate standard deviation for collection
#GUP_sd = GUP2.map(calculate_sd)
# median
#GUP_sd_median = GUP_sd.median()
#for polygon in polygons:
#    print('Greenup Polygon number '+str(polygon))
#    #subset polygon
#    tmp_poly = worldgrid.filterMetadata('polyID', 'equals', polygon).first().geometry()
#    #assetname
#    filename = 'GUP_'+str(polygon)
#    # export GUP to google drive
#    task = ee.batch.Export.image.toDrive(image=GUP_sd_median,description=filename,folder="curruspito_phenology",
#                                        scale=500,fileFormat='GeoTIFF',skipEmptyTiles=True,
#                                         crs = 'EPSG:4326',maxPixels = 1e13,region = tmp_poly)
    
#   task.start()

# ------- Peak
# subset Peak for given year
Peak = modis_phenoprod.select(['Peak_1','QA_Detailed_1'])
# only take pixels above a certain threshold level
Peak1 = Peak.map(mask_pixelsall(bstart=6,bend=7)).select('Peak_1')
# filter using mask where pixels have been stable
Peak2 = Peak1.map(mask_image)
# calculate standard deviation for collection
Peak_sd = Peak2.map(calculate_sd)
# median
#Peak_sd_median = Peak_sd.median()

#for polygon in polygons:
#    print('Peak Polygon number '+str(polygon))
    # subset polygon
#    tmp_poly = worldgrid.filterMetadata('polyID', 'equals', polygon).first().geometry()
    # assetname
#    filename= 'Peak_'+str(polygon)
    # export GUP to google drive
#    task = ee.batch.Export.image.toDrive(image=Peak_sd_median,description=filename,folder="curruspito_phenology",
#                                         fileFormat='GeoTIFF',skipEmptyTiles=True,maxPixels = 1e13,
#                                         region = tmp_poly)
#    task.start()

# ------- Dormancy # CHANGE RESOLUTION WRONG!!!!!!!
#Dormancy = modis_phenoprod.select(['Dormancy_1','QA_Detailed_1'])
# only take pixels above a certain threshold level
#Dormancy1 = Dormancy.map(mask_pixelsall(bstart=12,bend=13)).select('Dormancy_1')
# filter using mask where pixels have been stable
#Dormancy2 = Dormancy1.map(mask_image)
# calculate standard deviation for collection
#Dormancy_sd = Dormancy2.map(calculate_sd)
# median
#Dormancy_sd_median = Dormancy_sd.median()

#for polygon in polygons:
#    print('Dormancy Polygon number '+str(polygon))
    # subset polygon
#    tmp_poly = worldgrid.filterMetadata('polyID', 'equals', polygon).first().geometry()
    # assetname
#    filename= 'Dormancy_'+str(polygon)
    # export GUP to google drive
#    task = ee.batch.Export.image.toDrive(image=Dormancy_sd_median,description=filename,folder="curruspito_phenology",
#                                        scale=500,fileFormat='GeoTIFF',skipEmptyTiles=True,
#                                        crs = 'EPSG:4326',maxPixels = 1e13,region = tmp_poly)
#   task.start()


In [23]:
image = Peak_sd.first()

In [24]:
Map.addLayer(image)

In [22]:
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [18]:
Map.addLayer(image)

In [ ]:

    # ------- Season length
    season_length = Dormancy2.subtract(GUP2)
    # calculate standard deviation
    SL_sd = season_length.reduceNeighborhood(reducer='stdDev',kernel= ee.Kernel.square(6, 'pixels'),skipMasked =True)
    # create filename
    filename = 'SL_sd_'+str(year)
    # export Dormancy to google drive
    task = ee.batch.Export.image.toDrive(image=SL_sd,description=filename,folder="curruspito_phenology",
                                         scale=5565.974539663679,fileFormat='GeoTIFF',skipEmptyTiles=True,
                                         crs = 'EPSG:4326',maxPixels = 1e13)
    task.start()